朴素贝叶斯的一大特点就是特征的条件独立假设，但在现实情况下，**条件独立这个假设通常过于严格，在实际中很难成立**。特征之间的相关性限制了朴素贝叶斯的性能，所以本节笔者将继续介绍一种**放宽了条件独立假设**的贝叶斯算法——贝叶斯网络(Bayesian Network)。

[参考文章](https://mp.weixin.qq.com/s/WtJ62UNyhIihpGsspDC7qQ)

## 贝叶斯网络的直观例子：
> 假设我们需要通过头像真实性、粉丝数量和动态更新频率来判断一个微博账号是否为真实账号。各特征属性之间的关系如下图所示：
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200523225340921.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)
> 上图是一个**有向无环图(DAG)**，每个节点表示一个特征或者随机变量，特征之间的关系则是用箭头连线来表示，比如说动态的更新频率、粉丝数量和头像真实性都会对一个微博账号的真实性有影响，而头像真实性又对粉丝数量有一定影响。但仅有各特征之间的关系还不足以进行贝叶斯分析。除此之外，贝叶斯网络中每个节点还有一个与之对应的**概率表**。
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200523225715385.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)
> **第一张概率表表示的是账号是否真实**，因为该节点没有父节点，可以直接用先验概率来表示，表示账号真实与否的概率。**第二张概率表表示的是账号真实性对于头像真实性的条件概率**。比如说在头像为真实头像的条件下，账号为真的概率为0.88。在有了DAG和概率表之后，我们便可以利用贝叶斯公式进行定量的因果关系推断。假设我们已知某微博账号使用了虚假头像，那么其账号为虚假账号的概率可以推断为：
$$P(A=0|H=0)=\frac{P(H=0|A=0)\times P(A=0)}{P(H=0)}
        =\frac{P(H=0|A=0)\times P(A=0)}{P(H=0|A=0)P(A=0)+ P(H=0|A=1)P(A=1)}
        =\frac{0.88 \times 0.13}{0.88 \times 0.13 + 0.25 \times 0.87}
        =0.345$$
利用贝叶斯公式，我们可知在虚假头像的情况下其账号为虚假账号的概率为0.345。

一个贝叶斯网络通常由**有向无环图(DAG)**和节点对应的**概率表（CPT）**组成。

**当一个节点的父节点概率分布确定之后，该节点条件独立于其所有的非直接父节点。**

  一般来说，多变量非独立随机变量的联合概率分布计算公式如下：
$$P(x_1,x_2,....,x_n)=P(x_1)P(x_2|x_1)P(x_3|x_1,x_2).....P(x_n|x_1,x_2,...,x_{n-1})$$

我们以学生获得推荐信质量这样一个例子来进行贝叶斯网络的构造。具体有向图和概率表如下图所示：
![微信图片_20200522181618.png](https://img-blog.csdnimg.cn/20200523223400220.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)
例如，聪明（I=1）学生在容易(D=0)课程中得到A（G=0）的可能性是90%，得到B的可能性是8%，得到C的可能性是2%。聪明学生在困难课程中得到A的可能性就是为50%。

 考试难度、个人聪明与否都会影响到个人成绩，另外个人聪明与否也会影响到SAT分数，而个人成绩好坏会直接影响到推荐信的质量。

## 基于pgmpy的贝叶斯网络实现

**pgmpy**是一款基于Python的概率图模型包，主要包括贝叶斯网络和马尔可夫蒙特卡洛等常见概率图模型的实现以及推断方法

关于pgmpy的更多内容，可参考项目地址：[https://github.com/pgmpy/pgmpy](https://github.com/pgmpy/pgmpy)

### 构建模型框架，指定各变量之间的依赖关系

In [22]:
from pgmpy.factors import TabularCPD
from pgmpy.models import BayesianModel
student_model=BayesianModel([
    ('D','G'),
    ('I','G'),
    ('G','L'),
    ('I','S')
])
student_model

**CPD**是`conditional probability  distribution`的缩写,条件概率分布

### 构建各个节点和传入概率表并指定相关参数

In [23]:
diff_cpd=TabularCPD(
    variable='D',
    variable_card=2,
    values=[[0.6,0.4]]
)
intell_cpd=TabularCPD(
    variable='I',
    variable_card=2,
    values=[[0.7,0.3]]
)
grade_cpd=TabularCPD(
    variable='G', # 节点名称
    variable_card=3, # 节点取值个数
    values=[[0.3, 0.05, 0.9, 0.5], # 该节点的概率表
    [0.4, 0.25, 0.08, 0.3],
    [0.3, 0.7, 0.02, 0.2]],
    evidence=['I', 'D'], # 该节点的依赖节点
    evidence_card=[2, 2] # 依赖节点的取值个数
)
letter_cpd=TabularCPD(
    variable='L',
    variable_card=2,
    values=[
        [.1,.4,.99],
        [.9,.6,.01]
    ],
    evidence=['G'],
    evidence_card=[3]
)
SAT_cpd=TabularCPD(
    variable='S',
    variable_card=2,
    values=[
        [.95,.2],
        [.05,.8]
    ],
    evidence=['I'],
    evidence_card=[2]
)
grade_cpd

<TabularCPD representing P(G:3 | I:2, D:2) at 0x17d3db3dd88>

### 将包含概率表的各节点添加到模型中

In [24]:
student_model.add_cpds(diff_cpd,
                      intell_cpd,
                      grade_cpd,
                      letter_cpd,
                      SAT_cpd)
student_model.get_cpds('G')
# student_model.check_model()
# student_model.get_independencies()

<TabularCPD representing P(G:3 | I:2, D:2) at 0x17d3db3dd88>

获取模型各节点之间的依赖关系

In [25]:
student_model.get_independencies()

(D _|_ I, S | G)
(D _|_ L, G | I)
(D _|_ I, S, G | L)
(D _|_ L, G | S)
(G _|_ I, S, L | D)
(G _|_ L, D | I)
(G _|_ I, S, D | L)
(G _|_ I, L, D | S)
(I _|_ S, G, L | D)
(I _|_ S, D | G)
(I _|_ S, G, D | L)
(I _|_ L, G | S)
(L _|_ I, S, G | D)
(L _|_ G, D | I)
(L _|_ I, G, D | S)
(S _|_ I, L, G | D)
(S _|_ I, D | G)
(S _|_ I, G, D | L)

### 进行贝叶斯推断
  可见当聪明的学生碰上较简单的考试时，获得第一等成绩的概率高达0.9。


In [26]:
from pgmpy.inference import VariableElimination
student_infer=VariableElimination(student_model)
pro_G=student_infer.query(
    variables=['G'],
    evidence={'I':1,'D':0}
)
a=pro_G['G']
a.values

array([0.05, 0.25, 0.7 ])

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200523224427105.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)

## 基于pgmpy进行数据训练

### 生成模拟数据并以上述的学生推荐信的模型变量进行命名

In [27]:
import numpy as np 
import pandas as pd 
raw_data=np.random.randint(0,2,size=(1000,5))
data=pd.DataFrame(raw_data,columns=['D','I','G','L','S'])
data.loc[:3,:'G']

,D,I,G
0,0,1,1
1,0,0,0
2,1,1,0
3,1,1,0


### 基于数据进行模型训练

In [29]:
from pgmpy.estimators import MaximumLikelihoodEstimator,BaseEstimator 
model=BayesianModel([('D','G'),('I','G'),('G','L'),('I','S')])
print(model.nodes(),model.edges())
# 基于极大似然估计进行模型训练
model.fit(data, estimator_type=MaximumLikelihoodEstimator)
for cpd in model.get_cpds():
    print(f'CPD of {cpd.variable}')
    print(cpd)

['D', 'G', 'I', 'L', 'S'] [('D', 'G'), ('G', 'L'), ('I', 'G'), ('I', 'S')]


TypeError: Estimator object should be a valid pgmpy estimator.

![在这里插入图片描述](https://img-blog.csdnimg.cn/2020052322462186.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)